# Next basket recommendation - Markov chain sequential models

The task of next basket recommendation is to predict a content of customers basket at their future purchase.

Task has been assigned as a competition among the students who took the Algorithms of data mining course at Faculty of Information Technology @ Czech Technical University in Prague.

### Scoring function

The competition has been divided into 2 rounds. The difference between rounds is the used scoring function. 1st round scoring function is Jaccard similarity coefficient

$J(A,B) = \frac{ |A \cap B| }{ |A \cup B| }$,

where A is a real basket and B is a predicted one. Final score is a mean of $J(A,B)$ over all predictions. 2nd round scoring function is Generalized Jaccard similarity coefficient over multisets

$J_g(A,B) = \frac{ \sum_i min(a_i, b_i) }{ \sum_i max(a_i, b_i) }$,

where $A$ is a real basket and $B$ is a predicted one. $a_i$ (resp. $b_i$) is number of occurrences of $i$-th in $A$ (resp. $B$). Final score is a mean of $J_g(A,B)$ over all predictions. Generalized jaccard index takes into account cases where there is multiple occurrences of same products in one basket. Therefore the scoring function is stricter.

Score in 1st rnd: ~`0.14`

Position among others 1st rnd: `(not competitive enough)`

Score in 2nd rnd: TBA

Position among others 2nd rnd: TBA

### Disclaimer

Unfortunately I cannot provide the data for the problem as to avoid any legal issues. The data has been provided by a external company in collaboration with the university.

Even though I cannot provide the dataset I will still try to capture my thought process and all the ideas and examples will be shown.

## Dataset

We have been provided with 2 types of csv files.

First file contains 3 columns that together create a order history. The columns are `userid`, `date` and `itemids`. Columns are self-explanatory. `userid` is an ID number of an user, `date` is a date of a purchase and `itemids` is a space separated list of product IDs. Order history files come in 2 sizes. First, the smaller one, contains 1 700 000+ rows. Second file contains total of 8 800 000+ of data.

| userid 	 | date       	 | itemids           	 |
|----------|--------------|---------------------|
| 12345  	 | 1995-30-7  	 | 11111 22222       	 |
| 777777 	 | 2022-1-1   	 | 12314             	 |
| 425645 	 | 2020-12-31 	 | 45646 46511 11111 	 |

**(Markov chain models won't use item attributes)** Second file contains information about products. Each row represents one product. There is 25+ features about each product. Total number of products is ~1400.

| productid 	 | features.... 	 |
|-------------|----------------|
| 11111     	 | features     	 |
| 22222     	 | features     	 |


# Sequential Markov chain models

Having $U = \{u_1, \ldots , u_{|U|}\}$ a set of users and $I = \{i_1, \ldots , i_{|I|}\}$ a set of items, and $\mathcal{B^u}$ order history of user $u$, $\mathcal{B^u} = (B^u_1, \ldots, B^u_{t_u-1} )$, where $B^u_t \subseteq I$. The purchase history of all users is $\mathcal{B} = \{ \mathcal{B^{u_1}, \ldots, \mathcal{B^{u_{|U|}}}}\}$.

Models used will be discrete time Markov chains of order $1$. Meaning $p(X_t = x_t| X_{t-1} = x_{t-1}) = p(X_t = x_t| X_{t-1} = x_{t-1}, \ldots, X_0 = x_0)$. The transitions over baskets would create transition matrix extremely large as the shape would be $ 2^{|I|} \times 2^{|I|}$. Instead, we are gonna work with transitions between items, resulting in transition matrix of shape $ |I| \times |I| $.

## Data preprocessing

In [13]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

First import dataset

In [14]:
df = pd.read_csv('data/train100k.csv')
df.head()

,userid,date,itemids
0,7226385,2019-01-22,42203 41183 15823 39620
1,7226385,2019-02-12,54231 14939 39462
2,7226385,2019-03-11,15823 21028 39620 52846
3,7226385,2019-04-03,14939 39620 27542 21028 19353
4,7226385,2019-05-23,21028 21028 14939 15823


Since predictions will be based on transition matrix between items, we are gonna need map indices to product ids, to later lookup the matrix.

In [15]:
df = df.assign(itemids=lambda x: x['itemids'].apply(lambda l:list(set(l.split()))))
unique_products = df['itemids'].explode('itemids').unique()
id_to_prod = pd.Series(unique_products, name='itemid').astype({'itemid':'uint16'})
prod_to_id = pd.Series(id_to_prod.index.values, index=id_to_prod, name='id').astype({'id':'uint16'})

### Split data to train and validation

Last basket of each user will be used as a validation basket.

Rest will be used for estimating the transition matrix.

Also the last basket in training of each user will be needed to use later for predictions.

In [16]:
sample = df[:10000]

In [17]:
data_sample = (
    sample
        .groupby('userid')
        .apply(lambda x: x.iloc[:-1])
        .reset_index(drop=True)
)

last_data_sample = (
    data_sample
        .groupby('userid')
        .last()
        .reset_index()
    [['userid','itemids']]
)

test_sample = (
    sample
        .groupby('userid')
        .last()
        .reset_index()
    [['userid','itemids']]
)

### Basket size

For basket size mean and median of baskets is calculated for each user.

In [18]:
bskt_sz_sample = (
    data_sample.explode('itemids')
        .groupby(['userid','date'])
        .count()
        .reset_index()
        .groupby('userid')
    ['itemids']
        .agg(mean='mean', median='median')
)

### Estimating transition matrix

The probability $a_{l,i} = p(i \in B_t| l \in B_{t-1})$ that item $i$ is bought when $l$ was bought in previous basket will be estimated using maximum likelihood estimator. The estimation for $a_{l,i}$ given purchase history $\mathcal{B}$ is:

 $\hat{a}_{l,i} = \hat{p}(i \in B_t| l \in B_{t-1}) = \frac{\hat{p}(i \in B_t \land l \in B_{t-1})}{\hat{p}(l \in B_{t-1})} = \frac{|\{(B_t, B_{t-1}) : i \in B_t \land l \in B_{t-1}\}|}{|\{(B_t, B_{t-1}):l\in B_{t-1}\}|}$

**Disclaimer**

For personalised MC the estimation is done over user purchase history $B^u$.





In [19]:
def user_transition_count(user_order_history, transition_count, item_transition_count):
    if len(user_order_history) > 1:
        user_order_history['next_purchase'] = user_order_history['itemids'].shift(-1)
        for i in range(0, len(user_order_history)-1):
            for pid in user_order_history.iloc[i]['itemids']:
                item_transition_count[prod_to_id[int(pid)]] += 1
                for npid in user_order_history.iloc[i]['next_purchase']:
                    transition_count[prod_to_id[int(pid)],prod_to_id[int(npid)]] += 1


In [20]:
def transition_matrix(transition_count, item_transition_count):
    return np.divide(
        transition_count,
        item_transition_count[:, None],
        out=np.zeros_like(transition_count),
        where=item_transition_count[:, None]!=0
    )


### Predicting next basket

Probabilities of each item given transition matrix $B$ are calculated as follows:

$p(i \in B_t|B_{t-1}) = \frac{1}{|B_{t-1}|} \sum_{l \in B_{t-1}} p(i \in B_t| l \in B_{t-1})$.

Then top $k$ items with highest probabilities are selected as prediction, where $k$ is size of predicted basket.

In [21]:
def pred(userid, last_basket, trans_mat, basket_size):
    last_basket_size = len(last_basket)
    last_basket = map(int, last_basket)
    probs = np.zeros((len(unique_products),))
    for pid in last_basket:
        probs += np.ravel(trans_mat[prod_to_id[pid]])
    probs = probs / last_basket_size
    probs = pd.Series(probs, name='probs').sort_values(ascending=False)
    pred_bskt_size = int(np.floor(basket_size.loc[userid, 'mean']))
    indexes = probs.head(pred_bskt_size).index.to_list()
    return id_to_prod[indexes].to_list()

## Non-personalized Markov Chain model

Creation of transition matrix.

In [22]:
trans_cnt = np.zeros(shape=(len(unique_products),len(unique_products)),dtype=np.float32)
item_trans_cnt = np.zeros(shape=(len(unique_products),),dtype=np.int32)
data_sample.groupby('userid').apply(lambda x: user_transition_count(x[['date','itemids']],trans_cnt, item_trans_cnt))

trans_mat = transition_matrix(trans_cnt, item_trans_cnt)

### Prediction and evaluation

In [23]:
from common import dataframe_score

pred_df = last_data_sample.assign(
    pred= lambda dataf:dataf.apply(
        lambda row:pred(
            row['userid'],
            row['itemids'],
            trans_mat,
            bskt_sz_sample
        ),axis=1)
)

preds = test_sample.merge(pred_df[['userid','pred']], how='inner', on='userid').rename(columns={'itemids':'product_id'})
print('Non-PMC validation score: ',dataframe_score(preds))

Non-PMC validation score:  0.11597901222815876


## Personalised Markov chain model

The only difference from non-personalised MC is that own transition matrix is estimated for each user. Therefore transition cube is created. These matrices will be heavily sparse and for our smaller dataset of 100 000 users we would get 100 000 matrices of shape ~1400 by ~1400 which would result in nearly 1 TB of memory used. Therefore the matrices will be estimated in dense format and then transformed and stored in sparse CSR format.

In [24]:
def create_personalised_transition_cube(order_history, personalised_trans_cube):
    trans_cnt = np.zeros(shape=(len(unique_products),len(unique_products)),dtype=np.float32)
    item_trans_cnt = np.zeros(shape=(len(unique_products),),dtype=np.int32)
    user_transition_count(order_history, trans_cnt, item_trans_cnt)
    personalised_trans_cube[order_history['userid'].iloc[0]] = sp.csr_matrix(transition_matrix(trans_cnt, item_trans_cnt))


In [25]:
per_trans_cube = dict()
data_sample.groupby('userid').apply(lambda group: create_personalised_transition_cube(group, per_trans_cube))

""


### Prediction and evaluation

In [26]:
pred_df = last_data_sample.assign(
    pred= lambda dataf:dataf.apply(
        lambda row:pred(
            row['userid'],
            row['itemids'],
            per_trans_cube[row['userid']].todense(),
            bskt_sz_sample
        ),axis=1)
)

In [27]:
from common import dataframe_score

preds = test_sample.merge(pred_df[['userid','pred']], how='inner', on='userid').rename(columns={'itemids':'product_id'})
print('PMC validation score: ',dataframe_score(preds))

PMC validation score:  0.13462744041517455


## Summary

Median turned out to be better than mean for the size of basket.

Non-personalised MC

validation score ~`0.11`.

Personalised MC

validation score ~`0.134`.

Score on test data was `0.132`.

## Test data predictions and scaling to full dataset

Lastly we will scale to full dataset, predict on test data and format the data so that the evaluation server can process it.

In [28]:
def test_pred_format(test):
    test = test.rename(columns={'pred':'itemids'})
    test['itemids'] = test['itemids'].astype(str).str.replace(r'\[|\]|\,','', regex=True)
    return test

In [29]:
eval_data = pd.read_csv('data/test24k.csv').drop(columns=['itemids'])
pdf = df.merge(eval_data[['userid']], how='inner', on='userid')

data = (
    pdf
        .groupby('userid')
        .apply(lambda x: x.iloc[:-1])
        .reset_index(drop=True)
)

last_data = (
    pdf
        .groupby('userid')
        .last()
        .reset_index()
    [['userid','itemids']]
)

per_trans_cube = dict()
data.groupby('userid').apply(lambda group: create_personalised_transition_cube(group, per_trans_cube))

bskt_sz = (
    pdf.explode('itemids')
        .groupby(['userid','date'])
        .count()
        .reset_index()
        .groupby('userid')
    ['itemids']
        .agg(mean='median')
)

pred_df = last_data.assign(
    pred= lambda dataf:dataf.apply(
        lambda row:pred(
            row['userid'],
            row['itemids'],
            per_trans_cube[row['userid']].todense(),
            bskt_sz
        ),axis=1)
)

preds = eval_data.merge(pred_df[['userid','pred']], how='inner', on='userid')
test_pred_format(preds).to_csv('data/pmctest24k.csv', index=False)